In [1]:
# import kagglehub
# import shutil
# import os

# # Download latest version (this goes into KaggleHub cache folder)
# path = kagglehub.dataset_download("kazanova/sentiment140")

# print("KaggleHub download path:", path)

# # Your custom target folder
# custom_path = r"E:\github\data science\data-science\NLP"

# # Make sure the folder exists
# os.makedirs(custom_path, exist_ok=True)

# # Copy everything from KaggleHub cache folder to your custom folder
# for item in os.listdir(path):
#     s = os.path.join(path, item)
#     d = os.path.join(custom_path, item)
#     if os.path.isdir(s):
#         shutil.copytree(s, d, dirs_exist_ok=True)
#     else:
#         shutil.copy2(s, d)

# print("Dataset copied to:", custom_path)


In [2]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.optim import Adam

import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
df=pd.read_csv(r"E:\github\data science\data-science\NLP\training.1600000.processed.noemoticon.csv",encoding='latin-1',header=None)
df.columns=['target','ids','date','flag','user','text']
df.fillna('')
print(df.head())
print(df.describe())
print(df.info())

   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  
             target           ids
count  1.600000e+06  1.600000e+06
mean   2.000000e+00  1.998818e+09
std    2.000001e+00  1.935761e+08
min    0.000000e+00  1.467810e+09
25%    

In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{"should've", 'should', 'can', 'above', 'again', "we've", 'into', 'ma', "haven't", 'off', 'we', 'hasn', 'yourself', 'himself', 'm', 'other', 'which', 'a', 'yourselves', 'were', "mustn't", 'am', 'an', 'don', 'after', "it'd", 'this', 'how', 'only', 'between', 'been', 'me', 'such', "you'd", 'he', "shan't", 'its', 'about', 'had', 'couldn', 'because', 'having', 're', 'as', 'own', 'ourselves', 'it', "we'll", 'that', 'when', 't', "don't", 'the', "he'll", 'here', 'most', 'needn', 'our', 'herself', "she'll", 'your', 'hadn', "i've", 'not', "she'd", "she's", "needn't", "he's", 'him', 'each', 'shouldn', 'y', "i'm", 'whom', 'in', "you'll", "aren't", 'they', 'during', "they'll", "didn't", 'his', 'further', 'wasn', 'why', 'my', 'at', 'if', 'll', "doesn't", 'until', 'aren', 'do', 'once', "isn't", 'hers', "that'll", "it'll", 'didn', 'isn', 'was', "couldn't", 'for', 've', "wasn't", 'haven', 'doing', 'out', 'before', 'there', 'while', 'their', 'with', 'what', 'to', 'themselves', "wouldn't", 'no', "i'd", 

0->negative
1->positive


In [5]:
port_stem = PorterStemmer()
def stemming_tokenizer(text):
    stem_content=re.sub('[^a-zA-Z]',' ',text)
    stem_content=stem_content.lower()
    stem_content=stem_content.split()
    stem_content=[port_stem.stem(word) for word in stem_content if not word in stop_words]
    stem_content=' '.join(stem_content)
    return stem_content
print(f'example:{stemming_tokenizer("This is a sample text. We\'re testing the stemming_tokenizer function!")}')

example:sampl text test stem token function


In [6]:
# Apply stemming only to the first 1000 rows
df.loc[:99999, 'stemmed_text'] = df.loc[:99999, 'text'].apply(stemming_tokenizer)


In [7]:
df['stemmed_text'] = df['stemmed_text'].fillna("")
print(df.head(3))
print(df['stemmed_text'])


   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   

              user                                               text  \
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1    scotthamilton  is upset that he can't update his Facebook by ...   
2         mattycus  @Kenichan I dived many times for the ball. Man...   

                                        stemmed_text  
0  switchfoot http twitpic com zl awww bummer sho...  
1  upset updat facebook text might cri result sch...  
2  kenichan dive mani time ball manag save rest g...  
0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi fee

In [8]:
df['target']

0          0
1          0
2          0
3          0
4          0
          ..
1599995    4
1599996    4
1599997    4
1599998    4
1599999    4
Name: target, Length: 1600000, dtype: int64

In [9]:
x=df['stemmed_text'].values
y=df['target'].values
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ... '' '' '']


In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)
print(x_train.shape,x_test.shape)
print(x_train)
print(x_test)

(1280000, 54740) (320000, 54740)
  (19, 12780)	0.7176092382913959
  (19, 36214)	0.6964459642490884
  (28, 2284)	0.5415354953525429
  (28, 42873)	0.5367268420191788
  (28, 48446)	0.42651891231156536
  (28, 12948)	0.48656471488380093
  (39, 20619)	0.40899278316264215
  (39, 18209)	0.34365993182262305
  (39, 45512)	0.3975122824640948
  (39, 14911)	0.7460608151288552
  (49, 32490)	0.42743786614899953
  (49, 18034)	0.2849164052110756
  (49, 4670)	0.46282346059154206
  (49, 33477)	0.2537370060095376
  (49, 46295)	0.28138808372957713
  (49, 44051)	0.3827862173857738
  (49, 41829)	0.42247277200644034
  (49, 33250)	0.23096255921394548
  (112, 48190)	0.40720794895959234
  (112, 48871)	0.3881546311106826
  (112, 48746)	0.35695158269974836
  (112, 4709)	0.42707814971563535
  (112, 11356)	0.33935255121190244
  (112, 38836)	0.5084755021444388
  (135, 25982)	0.24369015746316824
  :	:
  (1279955, 49080)	0.2555360950718457
  (1279955, 51986)	0.28701569924242565
  (1279955, 9789)	0.2710722058388166
  (1

In [11]:
# use logistic regression
model=LogisticRegression(max_iter=100000)
model.fit(x_train,y_train)
x_train_pred=model.predict(x_train)

In [12]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print(cm)

Accuracy: 0.5626875
              precision    recall  f1-score   support

           0       1.00      0.12      0.22    159494
           4       0.53      1.00      0.70    160506

    accuracy                           0.56    320000
   macro avg       0.77      0.56      0.46    320000
weighted avg       0.77      0.56      0.46    320000

[[ 19554 139940]
 [     0 160506]]


In [13]:
import pickle
with open('sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle

# Example: Sample data - Replace with your real data loading
texts = [
    "I love this product",
    "This is the worst experience ever",
    "Not bad, could be better",
    "Excellent customer service and fast delivery"
]
labels = [1, 0, 1, 1]  # 1: Positive, 0: Negative

# Create and fit the vectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(texts)

# Train classifier
model = LogisticRegression()
model.fit(X, labels)

# Save both model and vectorizer
with open('sentiment_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved!")


Model and vectorizer saved!


In [15]:
# Load model and vectorizer
with open('sentiment_model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

# Simulate input text
text = "This product is great!"

# Your preprocessing function here (match training preprocessing)
processed_text = text.lower()

# Vectorize
vectorized_text = vectorizer.transform([processed_text])

# Predict
prediction = model.predict(vectorized_text)[0]
print(f"Prediction: {prediction}")


Prediction: 1
